In [ ]:
model_path = '/content/convnext_adam_best_model.pth' # Sesuaikan path jika disimpan di lokasi lain
try:
    model_adam.load_state_dict(torch.load(model_path, map_location=device))
    print(f"Model Adam berhasil dimuat dari {model_path}")
except FileNotFoundError:
    print(f"Error: Model Adam tidak ditemukan di {model_path}.")
    print("Pastikan file model ConvNeXt Adam terbaik ('convnext_adam_best_model.pth') sudah ada di lokasi yang benar.")
    exit() # Keluar jika model tidak ditemukan

# Set model ke mode evaluasi
model_adam.eval()

# Definisi transformasi untuk gambar testing
# Gunakan transformasi yang sama seperti saat evaluasi
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Fungsi untuk melakukan prediksi pada gambar tunggal
def predict_image(image_path, model, transform, device, classes=['Normal', 'Tuberculosis']):
    """
    Melakukan prediksi pada gambar tunggal menggunakan model yang sudah ditraining.
    """
    try:
        image = Image.open(image_path).convert('RGB')
    except FileNotFoundError:
        print(f"Error: File gambar tidak ditemukan di {image_path}")
        return None, None, None, None
    except Exception as e:
        print(f"Error saat memuat gambar {image_path}: {e}")
        return None, None, None, None

    # Terapkan transformasi
    image_tensor = transform(image).unsqueeze(0).to(device) # Tambahkan batch dimension

    # Lakukan prediksi
    with torch.no_grad():
        outputs = model(image_tensor)
        # Gunakan softmax untuk mendapatkan probabilitas
        probabilities = torch.softmax(outputs, dim=1)
        # Ambil prediksi kelas dengan probabilitas tertinggi
        confidence, predicted_class_idx = torch.max(probabilities, 1)

    predicted_label = classes[predicted_class_idx.item()]
    confidence_score = confidence.item()

    # Hitung probabilitas untuk kedua kelas
    prob_normal = probabilities[0, 0].item()
    prob_tb = probabilities[0, 1].item()


    return predicted_label, confidence_score, prob_normal, prob_tb

# Path ke folder yang berisi gambar untuk diuji
# Ganti dengan path folder Anda
test_folder_path = '/content/testing_image' # CONTOH: ganti dengan folder di Google Drive atau lokasi lain

# Pastikan folder test ada
if not os.path.exists(test_folder_path):
    print(f"Error: Folder test tidak ditemukan di {test_folder_path}")
    print("Buat folder ini atau ganti path ke folder gambar Anda.")
else:
    # Dapatkan daftar semua file gambar di folder test
    image_files = [f for f in os.listdir(test_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print(f"Tidak ada file gambar (.png, .jpg, .jpeg) ditemukan di folder {test_folder_path}")
    else:
        print(f"\nMelakukan prediksi pada {len(image_files)} gambar dari folder: {test_folder_path}")
        print("-" * 50)

        results = []
        # Proses setiap gambar di folder
        for image_file in tqdm(image_files, desc="Memproses Gambar"):
            image_path = os.path.join(test_folder_path, image_file)

            predicted_label, confidence_score, prob_normal, prob_tb = predict_image(
                image_path, model_adam, test_transform, device
            )

            if predicted_label is not None: # Jika gambar berhasil dimuat dan diproses
                 results.append({
                    'filename': image_file,
                    'predicted_label': predicted_label,
                    'confidence': confidence_score,
                    'probability_normal': prob_normal,
                    'probability_tb': prob_tb
                })
                 print(f"File: {image_file}")
                 print(f"  Prediksi: {predicted_label} (Confidence: {confidence_score:.4f})")
                 print(f"  Probabilitas Normal: {prob_normal:.4f}, Probabilitas TB: {prob_tb:.4f}")
                 print("-" * 20)

        # Tampilkan ringkasan hasil
        print("\n" + "="*50)
        print("RINGKASAN HASIL PREDIKSI FOLDER")
        print("="*50)

        if results:
            # Hitung jumlah prediksi per kelas
            predicted_normal_count = sum(1 for r in results if r['predicted_label'] == 'Normal')
            predicted_tb_count = sum(1 for r in results if r['predicted_label'] == 'Tuberculosis')

            print(f"Total gambar diproses: {len(results)}")
            print(f"Diprediksi Normal: {predicted_normal_count}")
            print(f"Diprediksi TB: {predicted_tb_count}")

            # Tampilkan detail hasil (opsional)
            print("\nDetail Hasil:")
            for res in results:
            print(f"  {res['filename']}: Prediksi={res['predicted_label']}, Confidence={res['confidence']:.4f}")

        else:
            print("Tidak ada gambar yang berhasil diproses.")